In [ ]:

input = read("day14.txt", String)

grid_size = [101, 103]
#grid_size = [11, 7]

struct Robot
    x::Int
    y::Int
    vx::Int
    vy::Int
end

function parse_item(x)
    m = match(r"p=(?<px>-?\d+),(?<py>-?\d+) v=(?<vx>-?\d+),(?<vy>-?\d+)", x)

    Robot(parse(Int, m["px"]),
        parse(Int, m["py"]),
        parse(Int, m["vx"]),
        parse(Int, m["vy"]))
end

function parse_input(input)
    input_list = split(input, "\n")
    [parse_item(item) for item in input_list]
end

robot_list = parse_input(input)

function step_robot(robot, seconds, grid_size)
    
    Robot(mod(robot.x + seconds*robot.vx, grid_size[1]),
    mod(robot.y + seconds*robot.vy, grid_size[2]),
    robot.vx,
    robot.vy)
    
end

function get_quad(robot, grid_size)
    # NW  NE
    # SW  SE
    
    x_border = (grid_size[1]-1)/2
    y_border = (grid_size[2]-1)/2
    
    if (robot.x == x_border) | (robot.y == y_border)
       return("border")
    elseif (robot.x < x_border) & (robot.y < y_border)
        return("NW")
    elseif (robot.x > x_border) & (robot.y < y_border)
        return("NE")
    elseif (robot.x < x_border) & (robot.y > y_border)
        return("SW")
    elseif (robot.x > x_border) & (robot.y > y_border)
        return("SE")
    else
        return("ERROR")
    end
    
end

#step_robot
stepped_robot_list = [step_robot(r,100,grid_size) for r in robot_list]

robot_quads = [get_quad(r, grid_size) for r in stepped_robot_list]
quads = Dict{String, Int}()
_ = [quads[q] = get(quads, q, 0) + 1 for q in robot_quads]
println("Puzzle 1: ", quads["NW"]*quads["NE"]*quads["SW"]*quads["SE"])

In [ ]:
# define RobotList so we can overload the print function to view it
struct RobotList
    robots::Vector{Robot}
    grid_size::Vector{Int}
end

# this is the string of robots
function grid_string(sj::RobotList)
    
    points = Dict{Vector{Int}, Int}()
    _ = [points[[p.x,p.y]] = get(points, [p.x,p.y], 0) + 1 for p in sj.robots]
    
    str = ""
    for y in 0:(sj.grid_size[2]-1)
        for x in (0:sj.grid_size[1]-1)
            str = str * string(get(points , [x,y], " "))
        end
        str = str * "\n"
    end

    return(str)
end

# this overloads the print function
Base.show(io::IO, r::RobotList) = print(io, grid_string(r))

# check if there is a loop (i.e. the robots all return to initial positions)
i = 0
initial_robotlist = RobotList(robot_list, grid_size)
current_robotlist = RobotList([step_robot(r,1,grid_size) for r in initial_robotlist.robots], grid_size)
i = i+1
while [[r.x,r.y] for r in current_robotlist.robots] != [[r.x,r.y] for r in initial_robotlist.robots]
    current_robotlist = RobotList([step_robot(r,1,grid_size) for r in current_robotlist.robots], grid_size)
    i = i + 1
    #println("i = ", start+i)
    #println(current_robotlist)
    #IJulia.clear_output(true)
    #sleep(0.05)
end
loop_length = i
# yes there is, with length:
println(loop_length)

# watch the robots progress because the loop is not so long
function watch_robots(start_n, max_iter, initial_robotlist, sleep_s)
start = start_n
current_robotlist = RobotList([step_robot(r,start,grid_size) for r in initial_robotlist.robots], initial_robotlist.grid_size)
for n in 1:max_iter
    current_robotlist = RobotList([step_robot(r,1,grid_size) for r in current_robotlist.robots], current_robotlist.grid_size)
    println("n = ", start + n)
    println(current_robotlist)
    IJulia.clear_output(true)
    sleep(sleep_s)
end
end

# uncomment to watch
# watch_robots(0,loop_length, initial_robotlist, 0.001)

# check the answer! (kept in file to avoid commiting)
solution = parse(Int, read("day14_solution.txt", String))
tree_robotlist = RobotList([step_robot(r,solution,grid_size) for r in initial_robotlist.robots], grid_size)
println(tree_robotlist)